In [2]:
from src.models.lightningmodel import LightningClassifierModelWrapper, modelling_choice, count_parameters
from src.preprocessing.data_loader import GeoEye1, compute_dataset_statistics, get_transforms, get_dataloaders

In [ ]:
trainer, lightning_model = modelling_choice(model_name="resnet18", max_epochs=40, pretrained=True)
mean, std = compute_dataset_statistics(root_dir="ipeo_hurricane_for_students", split="train")
torch.save(mean, "mean.pt")
torch.save(std, "std.pt")

In [ ]:
train_loader, val_loader, test_loader = get_dataloaders(root_dir="ipeo_hurricane_for_students", mean=mean, std=std, batch_size=100)
trainer.fit(lightning_model, train_loader, val_loader)
trainable_params, total_params = count_parameters(lightning_model)
print(f"Trainable parameters: {trainable_params}, Total parameters: {total_params}")    